<a href="https://colab.research.google.com/github/vKrRv/porta-trucks-detector/blob/main/truck-detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Porta 🚛
### Trucks Detector

## Setup

### Dependecies Operations

In [ ]:
from IPython.display import clear_output

> #### **Run this cell only once! After restart ignore it**

In [ ]:
# Install dependencies (Extracted from license-plate-detector)
# Run this once!
!pip install ultralytics
!pip install kagglehub
!pip install opencv-python
!pip install matplotlib
!pip install scikit-learn

clear_output()
print("Dependencies installed.")

Dependencies installed.


In [ ]:
import os
import kagglehub
import glob
import shutil
import yaml
from sklearn.model_selection import train_test_split
from ultralytics import YOLO

print("Libraries imported.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/ali/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Libraries imported.


In [ ]:
import torch

# Check for NVIDIA CUDA or AMD ROCm
if torch.cuda.is_available():
    device = torch.device("cuda")
    # Optional: Print specific GPU name to verify
    print(f"Device: {device} ({torch.cuda.get_device_name(0)})")

# Check for macOS Apple Silicon (Metal Performance Shaders)
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Device: {device}")

# Fallback to CPU
else:
    device = torch.device("cpu")
    print(f"Device: {device}")

# Example usage:
# model.to(device)
# data = data.to(device)


Device: cpu


In [ ]:
print(f"PyTorch version: {torch.__version__}")
if device.type == 'cuda':
    print(f"CUDA Version: {torch.version.cuda}")
    # If using AMD, this might show the HIP version or None depending on the build

PyTorch version: 2.10.0+cu128


### Dataset Operations

In [ ]:
# 1. Download the dataset
# IMPORTANT: Replace "andrewmvd/car-plate-detection" with YOUR vehicle dataset URL if needed.
# The current dataset is the one from your license-plate code.
dataset_url = "sakshamjn/vehicle-detection-8-classes-object-detection"
path = kagglehub.dataset_download(dataset_url)
print("Dataset downloaded to:", path)

100%|██████████| 346M/346M [00:17<00:00, 21.2MB/s] 

Extracting files...


Dataset downloaded to: /home/ali/.cache/kagglehub/datasets/sakshamjn/vehicle-detection-8-classes-object-detection/versions/1


In [ ]:
# 2. Setup Writable Directories (Since Kaggle input is read-only)
base_output_dir = '/kaggle/working/vehicle-classification-processed'

processed_images_path = os.path.join(base_output_dir, 'images')
processed_labels_path = os.path.join(base_output_dir, 'labels')

In [ ]:
import os
import shutil

base_output_dir = os.path.join(os.getcwd(), 'vehicle-classification-processed')

processed_images_path = os.path.join(base_output_dir, 'images')
processed_labels_path = os.path.join(base_output_dir, 'labels')

# --- Clean & Create Directories ---
if os.path.exists(base_output_dir):
    shutil.rmtree(base_output_dir)

# Create fresh subdirectories
for split in ['train', 'val']:
    os.makedirs(os.path.join(processed_images_path, split), exist_ok=True)
    os.makedirs(os.path.join(processed_labels_path, split), exist_ok=True)

print(f"Clean working directories created at: {base_output_dir}")

Clean working directories created at: /home/ali/Downloads/vehicle-classification-processed


In [ ]:
search_patterns = [
    os.path.join(path, "**", "*.jpg"),
    os.path.join(path, "**", "*.jpeg"),
    os.path.join(path, "**", "*.png")
]

all_images = []
for pattern in search_patterns:
    all_images.extend(glob.glob(pattern, recursive=True))

print(f"Found {len(all_images)} images in source.")

if len(all_images) == 0:
    raise FileNotFoundError("CRITICAL ERROR: No images found. Check the dataset path.")

Found 8218 images in source.


In [ ]:
train_files, val_files = train_test_split(all_images, test_size=0.2, random_state=42)

# 3. Move files function
def move_files(files, split_name):
    print(f"Processing {len(files)} files for {split_name}...")
    for image_file in files:
        # Move Image
        shutil.copy(image_file, os.path.join(processed_images_path, split_name))

        # Find and Move Label
        base_name = os.path.splitext(os.path.basename(image_file))[0]
        parent_dir = os.path.dirname(image_file)

        # Check standard locations for labels (YOLO usually expects .txt)
        potential_labels = [
            os.path.join(parent_dir, base_name + ".txt"),
            os.path.join(parent_dir, "labels", base_name + ".txt"),
            os.path.join(parent_dir, "..", "labels", base_name + ".txt")
        ]

        for label_path in potential_labels:
            if os.path.exists(label_path):
                shutil.copy(label_path, os.path.join(processed_labels_path, split_name))
                break

move_files(train_files, "train")
move_files(val_files, "val")

print("✅ Data setup complete.")
print(f"Train images: {len(os.listdir(os.path.join(processed_images_path, 'train')))}")
print(f"Val images: {len(os.listdir(os.path.join(processed_images_path, 'val')))}")


Processing 6574 files for train...
Processing 1644 files for val...
✅ Data setup complete.
Train images: 6574
Val images: 1644


In [ ]:
# --- STEP 4: Create YAML Configuration (Corrected for 8-Class Dataset) ---
# The dataset 'sakshamjn/vehicle-detection-8-classes-object-detection' has 8 specific classes.
# We must list ALL of them in the correct order so the model learns correctly.

yaml_content = f"""
path: {os.path.abspath(base_output_dir)}
train: images/train
val: images/val

# Number of classes
nc: 8

# Class names (Exact order from dataset source)
names: [
    'Motorcycle',
    'Auto',
    'Car',
    'Bus',
    'LCV',         # Light Commercial Vehicle
    'Truck',
    'Tractor',
    'Heavy Truck'  # Original label is 'Multi-Axle'
]
"""

with open("data.yaml", "w") as f:
    f.write(yaml_content)

print("data.yaml created with CORRECT 8-class configuration!")

data.yaml created with CORRECT 8-class configuration!


In [ ]:
# --- STEP 5: Start Training (M1 Stable Mode) ---
print(f"Starting training on {device}...")
from ultralytics import YOLO

# Load the model
model = YOLO('yolov8n.pt')

# Train with specific M1 fixes
results = model.train(
    data='data.yaml',
    epochs=50,
    imgsz=640,
    batch=16,           # Reduced to prevent memory pressure
    device=device,      # Use M1 GPU
    amp=False,         # CRITICAL: Disables Mixed Precision to fix NMS timeout
    workers=0,         # CRITICAL: Fixes data loading hangs on macOS
    project='vehicle_classifier',
    name='truker_train_run'
)

print("Training finished!")

> # Author: Mohammad Al-rebh